In [1]:
import pandas as pd

In [3]:
pf_fake = pd.read_csv(r"C:\Users\Lasse\Desktop\DTU\Kandidat\1. semester\Computational tools for data science\Project\02807_CompTools_Project\data\raw\PolitiFact_fake_news_content.csv")

In [11]:
pf_fake.head(3)

,id,title,text,url,top_img,authors,source,publish_date,movies,images,canonical_link,meta_data
0,Real_1-Webpage,Trump Just Insulted Millions Who Lost Everythi...,16.8k SHARES SHARE THIS STORY\n\nHillary Clint...,http://occupydemocrats.com/2016/09/27/trump-ju...,http://occupydemocrats.com/wp-content/uploads/...,"Brett Bose,Grant Stern,Steve Bernstein,Natalie...",http://occupydemocrats.com,{'$date': 1474934400000},NaN,http://occupydemocrats.com/wp-content/uploads/...,http://occupydemocrats.com/2016/09/27/trump-ju...,"{""generator"": ""Powered by Visual Composer - dr..."
1,Real_10-Webpage,Famous dog killed in spot she waited a year fo...,Famous dog killed in spot she waited a year fo...,http://rightwingnews.com/top-news/famous-dog-k...,http://rightwingnews.com/wp-content/uploads/20...,NaN,http://rightwingnews.com,{'$date': 1474948336000},NaN,http://rightwingnews.com/wp-content/uploads/20...,http://rightwingnews.com/top-news/famous-dog-k...,"{""googlebot"": ""noimageindex"", ""og"": {""site_nam..."
2,Real_100-Webpage,House oversight panel votes Clinton IT chief i...,Story highlights The House Oversight panel vot...,http://cnn.it/2deaH2d,http://i2.cdn.cnn.com/cnnnext/dam/assets/16091...,"Tom Lobianco,Deirdre Walsh",http://cnn.it,NaN,NaN,http://i2.cdn.cnn.com/cnnnext/dam/assets/17050...,http://www.cnn.com/2016/09/22/politics/bryan-p...,"{""description"": ""Members of the House Oversigh..."


## Creating a TF-IDF representation.

In [19]:
# Concatenate title and text
pf_fake["full_text"] = pf_fake["title"] + " " + pf_fake["text"]

In [50]:
import pandas as pd
from collections import Counter
import math
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# Download the stopwords dataset
nltk.download('stopwords')

def tokenize(text, stemmer, stop_words):
    """Tokenize text, stem, and remove stop words."""
    tokens = re.findall(r'\b\w+\b', text.lower())
    return [stemmer.stem(token) for token in tokens if token not in stop_words]

def compute_tfidf(df, column_name):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))

    # Tokenize, stem and compute term frequency for each document
    df['tf'] = df[column_name].apply(lambda x: Counter(tokenize(x, stemmer, stop_words)))

    # Create a list of all unique words in the corpus
    all_unique_words = list(set(word for tf in df['tf'] for word in tf.keys()))

    # Compute the number of documents in which each word appears
    word_document_count = Counter(word for tf in df['tf'] for word in tf.keys())

    # Compute the total number of documents
    total_documents = len(df)

    # Compute inverse document frequency for each word
    idf = {word: math.log(total_documents / count) for word, count in word_document_count.items()}

    # Compute TF-IDF for each word in each document and ensure that all words in the corpus are represented
    def calculate_tfidf(tf):
        return {word: tf.get(word, 0)/sum(tf.values()) * idf[word] for word in all_unique_words}

    df['tfidf'] = df['tf'].apply(calculate_tfidf)

    return df['tfidf']


tfidf_scores = compute_tfidf(pf_fake, 'full_text')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lasse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0      {'mud': 0.0, 'seriou': 0.0, 'beyond': 0.0, 'fl...
1      {'mud': 0.0, 'seriou': 0.0, 'beyond': 0.0, 'fl...
2      {'mud': 0.0, 'seriou': 0.0, 'beyond': 0.0, 'fl...
3      {'mud': 0.0, 'seriou': 0.0, 'beyond': 0.0, 'fl...
4      {'mud': 0.0, 'seriou': 0.0, 'beyond': 0.0, 'fl...
                             ...                        
115    {'mud': 0.0, 'seriou': 0.0, 'beyond': 0.0, 'fl...
116    {'mud': 0.0, 'seriou': 0.0, 'beyond': 0.0, 'fl...
117    {'mud': 0.0, 'seriou': 0.002206360447568847, '...
118    {'mud': 0.0, 'seriou': 0.0, 'beyond': 0.0, 'fl...
119    {'mud': 0.0, 'seriou': 0.0, 'beyond': 0.0, 'fl...
Name: tfidf, Length: 120, dtype: object


In [51]:
print(len(tfidf_scores[0]))

6364


In [36]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

def dataframe_to_tfidf(df, column_name):
    """
    Convert a text corpus in a pandas DataFrame to its TF-IDF representation.
    
    Parameters:
    - df (pd.DataFrame): The pandas DataFrame containing the text corpus.
    - column_name (str): The column in the DataFrame containing the text corpus.
    
    Returns:
    - tfidf_matrix (sparse matrix): The TF-IDF representation of the text corpus.
    - tfidf_feature_names (list): List of feature names (words) for the TF-IDF matrix.
    """
    
    # Initialize the TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.85)
    
    # Fit and transform the text corpus
    tfidf_matrix = tfidf_vectorizer.fit_transform(df[column_name])
    
    # Get feature names
    tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
    
    return tfidf_matrix, tfidf_feature_names

# Example Usage:
# df = pd.DataFrame({'text': ["This is the first document.", 
#                             "This document is the second document.", 
#                             "And this is the third one."]})
tfidf_matrix, tfidf_feature_names = dataframe_to_tfidf(pf_fake, 'full_text')
print(tfidf_matrix.shape)
# print(tfidf_feature_names)


(120, 8777)
